# Import modules

In [127]:
!pip install graphene
import graphene
import json
import sys, io
import pandas as pd
import numpy as np
import importlib
import os
import pathlib
import requests
import torch

bundle_root = pathlib.Path(os.environ.get('LABS_BUNDLE_ROOT', '/labs'))
sys.path.append(str(bundle_root / 'functions/index_by_nationality'))

In [84]:
query_metadata = """query metadata {nationalityList, jobList}"""
query_data = """query data ($nationality: String!, $job: String!)
            {movieList(nationality: $nationality, job: $job)
            {
                title 
                revenue
                }}"""
query_introspect = """{
  __schema {
    types {
      name
    }
  }
}"""
query_playground = """{"operationName":"IntrospectionQuery","variables":{},"query":"query IntrospectionQuery {\n  __schema {\n    queryType {\n      name\n    }\n    mutationType {\n      name\n    }\n    subscriptionType {\n      name\n    }\n    types {\n      ...FullType\n    }\n    directives {\n      name\n      description\n      locations\n      args {\n        ...InputValue\n      }\n    }\n  }\n}\n\nfragment FullType on __Type {\n  kind\n  name\n  description\n  fields(includeDeprecated: true) {\n    name\n    description\n    args {\n      ...InputValue\n    }\n    type {\n      ...TypeRef\n    }\n    isDeprecated\n    deprecationReason\n  }\n  inputFields {\n    ...InputValue\n  }\n  interfaces {\n    ...TypeRef\n  }\n  enumValues(includeDeprecated: true) {\n    name\n    description\n    isDeprecated\n    deprecationReason\n  }\n  possibleTypes {\n    ...TypeRef\n  }\n}\n\nfragment InputValue on __InputValue {\n  name\n  description\n  type {\n    ...TypeRef\n  }\n  defaultValue\n}\n\nfragment TypeRef on __Type {\n  kind\n  name\n  ofType {\n    kind\n    name\n    ofType {\n      kind\n      name\n      ofType {\n        kind\n        name\n        ofType {\n          kind\n          name\n          ofType {\n            kind\n            name\n            ofType {\n              kind\n              name\n              ofType {\n                kind\n                name\n              }\n            }\n          }\n        }\n      }\n    }\n  }\n}\n"}"""

In [112]:
query_data.replace('\n', '')

'query data ($nationality: String!, $job: String!)            {movieList(nationality: $nationality, job: $job)            {                title                 revenue                }}'

In [113]:
input_query_data = json.dumps({'variables': {'nationality': 'English', 'job': 'Director'}, 'query':query_data.replace('\n', '')})
input_query_metadata = json.dumps({'query':query_metadata})

In [114]:
input_query_data#.replace('\\n', '').replace('\n', '')

'{"variables": {"nationality": "English", "job": "Director"}, "query": "query data ($nationality: String!, $job: String!)            {movieList(nationality: $nationality, job: $job)            {                title                 revenue                }}"}'

In [120]:
input_json = input_query_data
try:
    input_dict = json.loads(input_json.replace('\\n', '').replace('\n', ''))
except ValueError as e:
    print('Input must be json.' )

In [121]:
input_dict

{'variables': {'nationality': 'English', 'job': 'Director'},
 'query': 'query data ($nationality: String!, $job: String!)            {movieList(nationality: $nationality, job: $job)            {                title                 revenue                }}'}

# Test handler functions

In [128]:
import handler 
importlib.reload(handler);

In [129]:
%%time
handler.handle(query_data)

query data ($nationality: String!, $job: String!)
            {movieList(nationality: $nationality, job: $job)
            {
                title 
                revenue
                }}


JSONDecodeError: Expecting value: line 1 column 1 (char 0)

In [46]:
with open(str(bundle_root / 'functions/index_by_nationality/schema.json'), 'w') as fp:
    json.dump(handler.schema.introspect(), fp)

# Deploy function

### cheat sheet

In [6]:
# labs bundle deploy staging (deploy to staging)
# labs bundle build classify_name (build locally for debugging)
# labs function run sunburst -d '{"date_range": [20180201,20180401], "max_level":4, "threshold":1000, "operating_system": "all"}' (example local call)

In [ ]:
#labs bundle build classify_name && docker run -it --rm --name function_test -p 5000:5000 classify_name python index.py

# Test function 

In [47]:
function_url = 'https://labs.cloud.contiamo.com/b0a002bf-d166-5ccc-a552-ebe89882cb90/v1/function/be852eaa-c701-4f73-8ed7-aceb6dd4aeb8/index_by_nationality'

In [105]:
%%time
deployed_output = requests.post(function_url,
                                data=input_query_data).json()

CPU times: user 18.7 ms, sys: 3.9 ms, total: 22.6 ms
Wall time: 247 ms


In [106]:
deployed_output

{'movieList': [{'title': 'Madagascar 3: Europes Most Wanted',
   'revenue': 746921274},
  {'title': 'Mamma Mia!', 'revenue': 609841637},
  {'title': 'Madagascar: Escape 2 Africa', 'revenue': 603900354}]}

### Compare

In [165]:
output['sunburst_data'] == deployed_output['sunburst_data']

True